In [1]:
import numpy as np
import pandas as pd

In [3]:
import os
os.getcwd()

'C:\\Users\\suare\\myscripts\\tdp\\amanecer\\MTE\\notebooks'

In [4]:
os.chdir("../data")

In [6]:
df = pd.read_csv('pesajes-01-01-2019-31-12-2020_anonimizado.csv' )
df['fecha'] = pd.to_datetime(df['fecha'])

df_precios = pd.read_csv('precios_predio.csv')
df = df.fillna('No especificado')

In [60]:
df[df.legacyId=="2026"]

,fecha,etapa,bolson,camion,peso,cartonero,legacyId,predio,material,tipoCartonero
74587,2020-10-23,AVELLANEDA,28.0,AD473FK,27,COMPA308,2026,CORTEJARENA,Mezcla B,RA
75227,2020-10-30,AVELLANEDA,28.0,AD333TS,45,COMPA308,2026,CORTEJARENA,Mezcla B,RA
76928,2020-11-06,AVELLANEDA,28.0,AD473FA,21,COMPA308,2026,CORTEJARENA,Mezcla B,RA
78082,2020-11-13,AVELLANEDA,28.0,AD410GN,58,COMPA308,2026,CORTEJARENA,Mezcla B,RA
78945,2020-11-16,AVELLANEDA,28.0,AD333TU,36,COMPA308,2026,CORTEJARENA,Mezcla B,RA
81160,2020-11-27,AVELLANEDA,28.0,AD333TW,21,COMPA308,2026,CORTEJARENA,Mezcla B,RA
81749,2020-11-30,AVELLANEDA,28.0,AD473GF,33,COMPA308,2026,CORTEJARENA,Mezcla B,RA
82942,2020-12-04,AVELLANEDA,28.0,No especificado,36,COMPA308,2026,CORTEJARENA,Mezcla B,RA
84134,2020-12-10,AVELLANEDA,28.0,No especificado,29,COMPA308,2026,CORTEJARENA,Mezcla B,RA
84353,2020-12-11,AVELLANEDA,28.0,No especificado,46,COMPA308,2026,CORTEJARENA,Mezcla B,RA


In [12]:
df_precios

,Sede,Material,Precio RA,Precio LE
0,AVELLANEDA,Mezcla B,25,15
1,AVELLANEDA,Papel Blanco B,20,20
2,AVELLANEDA,Vidrio B,30,40
3,AVELLANEDA,Chatarra,15,30
4,AVELLANEDA,TELA,35,25


In [9]:
def agregar_tipoCartonero(df):
    df_tipoCartonero = df.copy()
    df_tipoCartonero["tipoCartonero"] = "RA"
    indices_LE = df_tipoCartonero[df_tipoCartonero["legacyId"].str.startswith("LE")].index
    indices_none = df_tipoCartonero[df_tipoCartonero["legacyId"] == "No especificado"].index

    df_tipoCartonero.loc[indices_LE, "tipoCartonero"] = "LE"
    df_tipoCartonero.loc[indices_none, "tipoCartonero"] = "No especificado"

    return df_tipoCartonero

df = agregar_tipoCartonero(df)

In [19]:
def retornar_df_pagos(df_filtrado, df_precios):
    df_pagos = df_filtrado.copy()
    df_pagos["precio/kg"] = 0
    df_pagos["precio"] = 0
    for index, row in df_precios.iterrows():
        sede = row["Sede"]
        material = row["Material"]
#         tipoCartonero = row["tipoCartonero"]

        indices_LE = df_pagos[(df_pagos["predio"] == sede) & (df_pagos["material"] == material) & (df_pagos["tipoCartonero"] == "LE")].index
        indices_RA = df_pagos[(df_pagos["predio"] == sede) & (df_pagos["material"] == material) & (df_pagos["tipoCartonero"] == "RA")].index
#         indices_NE = df_pagos[(df_pagos["predio"] == sede) & (df_pagos["material"] == material) & (df_pagos["tipoCartonero"] == tipoCartonero)].index

        precio_LE = row["Precio LE"]
        precio_RA = row["Precio RA"]
        df_pagos.loc[indices_LE, "precio/kg"] = precio_LE
        df_pagos.loc[indices_RA, "precio/kg"] = precio_RA
        
    df_pagos["precio"] = df_pagos["precio/kg"]*df_pagos["peso"] 
    return df_pagos
        
df_pagos = retornar_df_pagos(df, df_precios)
df_pagos

,fecha,etapa,bolson,camion,peso,cartonero,legacyId,predio,material,tipoCartonero,precio/kg,precio
0,2019-12-02,R26,55.0,HVS 858,29,COMPA1987,3633,CORTEJARENA,No especificado,RA,0,0
1,2019-12-02,R26,28.0,HVS 858,93,COMPA687,3599,CORTEJARENA,No especificado,RA,0,0
2,2019-12-02,R26,61.0,HVS 858,35,COMPA2144,3638,CORTEJARENA,No especificado,RA,0,0
3,2019-12-02,R26,33.0,HVS 858,27,COMPA972,3608,CORTEJARENA,No especificado,RA,0,0
4,2019-12-02,R26,30.0,HVS 858,45,COMPA693,3603,CORTEJARENA,No especificado,RA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
89945,2020-12-30,E13,92.0,No especificado,57,COMPA669,5526,SAAVEDRA,Mezcla B,RA,0,0
89946,2020-12-30,E13,152.0,No especificado,76,COMPA1240,4972,SAAVEDRA,Mezcla B,RA,0,0
89947,2020-12-30,E13,224.0,No especificado,33,COMPA597,6376,SAAVEDRA,Mezcla B,RA,0,0
89948,2020-12-30,E13,224.0,No especificado,46,COMPA597,6376,SAAVEDRA,Mezcla B,RA,0,0


In [36]:
def pago_por_compa(df_filtrado, df_precios):
    df_pagos = retornar_df_pagos(df, df_precios)
    df_pagos = df_pagos.groupby(['cartonero']).sum('precio')
    return df_pagos[['precio']]

pago_compa = pago_por_compa(df, df_precios)
pago_compa.loc["COMPA308"]

precio    14280
Name: COMPA308, dtype: int64

In [34]:
pago_compa

,precio
cartonero,
COMPA0,0
COMPA1,0
COMPA10,0
COMPA100,0
COMPA1000,0
...,...
COMPA995,0
COMPA996,0
COMPA997,0


In [55]:
df[df.predio=="AVELLANEDA"]

,fecha,etapa,bolson,camion,peso,cartonero,legacyId,predio,material,tipoCartonero
84496,2020-12-11,AVELLANEDA,28.0,No especificado,115,COMPA308,2026,AVELLANEDA,TELA,RA
84497,2020-12-11,AVELLANEDA,28.0,No especificado,200,COMPA308,2026,AVELLANEDA,TELA,RA
84498,2020-12-11,AVELLANEDA,28.0,No especificado,93,COMPA308,2026,AVELLANEDA,TELA,RA
84499,2020-12-11,AVELLANEDA,46.0,No especificado,240,COMPA2104,6308,AVELLANEDA,TELA,RA
84500,2020-12-11,AVELLANEDA,46.0,No especificado,160,COMPA2104,6308,AVELLANEDA,TELA,RA
84501,2020-12-11,AVELLANEDA,47.0,No especificado,120,COMPA773,6501,AVELLANEDA,TELA,RA
84502,2020-12-11,AVELLANEDA,47.0,No especificado,110,COMPA773,6501,AVELLANEDA,TELA,RA
84503,2020-12-11,AVELLANEDA,90.0,No especificado,130,COMPA1660,2014,AVELLANEDA,TELA,RA
84504,2020-12-11,AVELLANEDA,92.0,No especificado,68,COMPA123,2049,AVELLANEDA,TELA,RA
84505,2020-12-11,AVELLANEDA,52.0,No especificado,195,COMPA1836,7244,AVELLANEDA,TELA,RA


In [56]:
def pago_individual(df_pagos, cartonero):
    return df_pagos.loc[cartonero]

pago_ind = pago_individual(pago_compa, "COMPA308")

In [59]:
pago_ind[0]

14280

In [15]:
def pago_por_predio(df, df_precios):
    df_pagos = retornar_df_pagos(df, df_precios)
    df_pagos = df_pagos.groupby('predio').sum('precio')
    return df_pagos[['precio']]

pago_por_predio(df, df_precios)

,precio
predio,
AVELLANEDA,199965
BARRACAS,17130320
CORTEJARENA,0
No especificado,0
SAAVEDRA,14272650


In [16]:
def pago_total(df, df_precios):
    df_pagos = retornar_df_pagos(df, df_precios)
    df_pagos = df_pagos['precio'].sum()
    return df_pagos

pago_total(df, df_precios)

31602935